In [1]:
import pandas as pd

model1 = pd.read_csv("submission_catboost_impute median with columns dropped and non oversampling and catboost and freq encoder.csv")
model2 = pd.read_csv("submission_lgbm_impute median with columns dropped and non oversampling and catboost and freq encoder.csv")
model3 = pd.read_csv("submission_catboost_impute median with columns dropped and oversampling and catboost and freq encoder.csv")

In [ ]:
# Model Voting for two-column submissions (ID + target)
import numpy as np

# Infer column names: first is ID, second is target
id_col = model1.columns[0]
target_col = model1.columns[1]

# Sanity check: ensure columns match across files
assert list(model2.columns)[:2] == [id_col, target_col], "model2 columns mismatch"
assert list(model3.columns)[:2] == [id_col, target_col], "model3 columns mismatch"

# Merge on ID to guarantee alignment
df = model1[[id_col, target_col]].merge(
    model2[[id_col, target_col]].rename(columns={target_col: f"{target_col}_m2"}),
    on=id_col, how="inner"
).merge(
    model3[[id_col, target_col]].rename(columns={target_col: f"{target_col}_m3"}),
    on=id_col, how="inner"
 )

# Compute soft-vote average
p_avg = (df[target_col].astype(float) + df[f"{target_col}_m2"].astype(float) + df[f"{target_col}_m3"].astype(float)) / 3.0

# Create outputs
out_prob = pd.DataFrame({id_col: df[id_col], target_col: p_avg})
out_labels = pd.DataFrame({id_col: df[id_col], target_col: (p_avg >= 0.5).astype(int)})

# Save files
out_prob.to_csv("submission_model_voting_prob.csv", index=False)
out_labels.to_csv("submission_model_voting_labels.csv", index=False)

print("Saved:", "submission_model_voting_prob.csv", "and", "submission_model_voting_labels.csv")